In [1]:
import pandas as pd

# Load datasets with specific data types
df_21 = pd.read_csv('FIFA21_official_data.csv', dtype={'Preferred Foot': str, 'Work Rate': str})
df_22 = pd.read_csv('FIFA22_official_data.csv', dtype={'Preferred Foot': str, 'Work Rate': str})
df_23 = pd.read_csv('FIFA23_official_data.csv', dtype={'Preferred Foot': str, 'Work Rate': str})

# Function to handle missing values and specific column transformations
def handle_missing_values_and_transform(df):
    # Handle missing values for numeric columns
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
    df[numeric_cols] = df[numeric_cols].apply(lambda x: x.fillna(x.median()), axis=0)
    
    # Handle missing values for categorical columns
    categorical_cols = df.select_dtypes(include=['object']).columns
    for col in categorical_cols:
        mode_value = df[col].mode()[0] if not df[col].mode().empty else 'Unknown'
        df[col] = df[col].fillna(mode_value)
    
    # Clean up 'Best Overall Rating' column if it exists
    if 'Best Overall Rating' in df.columns:
        df['Best Overall Rating'] = df['Best Overall Rating'].astype(str).str.extract(r'(\d+)').astype(float)
    
    return df

# Apply function to datasets
df_21 = handle_missing_values_and_transform(df_21)
df_22 = handle_missing_values_and_transform(df_22)
df_23 = handle_missing_values_and_transform(df_23)

# Combine datasets
common_columns = list(set(df_21.columns) & set(df_22.columns) & set(df_23.columns))
df_21 = df_21[common_columns]
df_22 = df_22[common_columns]
df_23 = df_23[common_columns]

combined_df = pd.concat([df_21, df_22, df_23], ignore_index=True)

# Check for remaining missing values
def check_missing_values(df, name):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        print(f"Remaining missing values in {name} Data:")
        print(missing)
    else:
        print(f"No missing values in {name} Data.")

# Check for missing values in the combined dataset
check_missing_values(combined_df, "Combined")

# Print the total number of rows and columns in the combined dataset
total_rows = combined_df.shape[0]
total_columns = combined_df.shape[1]
print("Total rows in the combined dataset:", total_rows)
print("Total columns in the combined dataset:", total_columns)


No missing values in Combined Data.
Total rows in the combined dataset: 51478
Total columns in the combined dataset: 28


In [9]:
# Step 6: Convert data types

# Display sample values before conversion
print("Sample Values Before Conversion:")
print(combined_df[['Value', 'Wage', 'Release Clause']].head())

# Ensure the columns are strings
combined_df['Value'] = combined_df['Value'].astype(str)
combined_df['Wage'] = combined_df['Wage'].astype(str)
combined_df['Release Clause'] = combined_df['Release Clause'].astype(str)

# Convert 'Value', 'Wage', and 'Release Clause' to numeric after removing non-numeric characters
combined_df['Value'] = combined_df['Value'].str.replace('€', '').str.replace('M', 'e6').str.replace('K', 'e3').astype(float)
combined_df['Wage'] = combined_df['Wage'].str.replace('€', '').str.replace('K', 'e3').astype(float)
combined_df['Release Clause'] = combined_df['Release Clause'].str.replace('€', '').str.replace('M', 'e6').str.replace('K', 'e3').astype(float)

# Display data types and sample values after conversion
print("\nData Types After Conversion:")
print(combined_df[['Value', 'Wage', 'Release Clause']].dtypes)

print("\nSample Values After Conversion:")
print(combined_df[['Value', 'Wage', 'Release Clause']].head())


# Step 6b: 
# Display data types before conversion
print("Data Types Before Conversion:")
print(combined_df[['Joined', 'Contract Valid Until']].dtypes)

# Display sample values before conversion
print("\nSample Values Before Conversion:")
print(combined_df[['Joined', 'Contract Valid Until']].head())

# Convert 'Joined' and 'Contract Valid Until' to datetime
combined_df['Joined'] = pd.to_datetime(combined_df['Joined'], errors='coerce')
combined_df['Contract Valid Until'] = pd.to_datetime(combined_df['Contract Valid Until'], format='%Y', errors='coerce')

# Display data types after conversion
print("\nData Types After Conversion:")
print(combined_df[['Joined', 'Contract Valid Until']].dtypes)

# Display sample values after conversion
print("\nSample Values After Conversion:")
print(combined_df[['Joined', 'Contract Valid Until']].head())

# Function to convert height to centimeters
def convert_height(height):
    if isinstance(height, str):
        if "'" in height:  # Format: 5'11"
            feet, inches = height.split("'")
            return int(feet) * 30.48 + int(inches.replace('"', '')) * 2.54
        elif "cm" in height:  # Format: 180cm
            return float(height.replace('cm', ''))
    return float('nan')  # Handle unexpected formats

# Function to convert weight to kilograms
def convert_weight(weight):
    if isinstance(weight, str):
        if "lbs" in weight:  # Format: 190lbs
            return int(weight.replace('lbs', '')) * 0.453592
        elif "kg" in weight:  # Format: 80kg
            return float(weight.replace('kg', ''))
    return float('nan')  # Handle unexpected formats

# Display sample values before conversion
print("Sample Values Before Conversion:")
print(combined_df[['Height', 'Weight']].head())

# Convert 'Height' to centimeters and 'Weight' to kilograms
combined_df['Height'] = combined_df['Height'].apply(convert_height)
combined_df['Weight'] = combined_df['Weight'].apply(convert_weight)

# Display data types and sample values after conversion
print("\nData Types After Conversion:")
print(combined_df[['Height', 'Weight']].dtypes)

print("\nSample Values After Conversion:")
print(combined_df[['Height', 'Weight']].head())

Sample Values Before Conversion:
        Value      Wage  Release Clause
0  31500000.0  115000.0      64600000.0
1  87000000.0  370000.0     161000000.0
2  63000000.0  195000.0     124400000.0
3  50500000.0  290000.0     103500000.0
4  22000000.0   41000.0      46200000.0

Data Types After Conversion:
Value             float64
Wage              float64
Release Clause    float64
dtype: object

Sample Values After Conversion:
        Value      Wage  Release Clause
0  31500000.0  115000.0      64600000.0
1  87000000.0  370000.0     161000000.0
2  63000000.0  195000.0     124400000.0
3  50500000.0  290000.0     103500000.0
4  22000000.0   41000.0      46200000.0
Data Types Before Conversion:
Joined                  datetime64[ns]
Contract Valid Until    datetime64[ns]
dtype: object

Sample Values Before Conversion:
      Joined Contract Valid Until
0 2020-09-25           2022-01-01
1 2015-08-30           2023-01-01
2 2020-01-30           2025-01-01
3 2019-07-12           2024-01-01
4 2020

In [10]:
combined_df.to_csv('combinedCleanedData2.csv', index=False)